In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [3]:
users = df[['person']].drop_duplicates().copy()

In [4]:
users_con_vistas = set(df.loc[df['event'] == 'viewed product', 'person'])
users['vio_productos'] = users['person'].isin(users_con_vistas)

users_con_checkouts = set(df.loc[df['event'] == 'checkout', 'person'])
users['tiene_checkouts'] = users['person'].isin(users_con_checkouts)

#Como hay muy pocas leads, de momento no las agrego
#users_con_leads = set(df.loc[df['event'] == 'lead', 'person'])
#users['tiene_leads'] = users['person'].isin(users_con_leads)

users_con_conversions = set(df.loc[df['event'] == 'conversion', 'person'])
users['tiene_conversions'] = users['person'].isin(users_con_conversions)
users.set_index('person', inplace=True)

In [5]:
users.head()

,vio_productos,tiene_checkouts,tiene_conversions
person,,,
4886f805,True,True,False
ad93850f,True,True,False
0297fc1e,True,True,False
2d681dd8,True,True,False
cccea85e,True,True,False


In [6]:
cant_prod_vistos = df.loc[df['event'] == 'viewed product']
cant_prod_vistos = cant_prod_vistos.groupby('person')['event'].value_counts().unstack()
cant_prod_vistos.rename(columns={'viewed product':'cant_prod_vistos'}, inplace=True)
users = users.join(cant_prod_vistos)
users.fillna({'cant_prod_vistos':0}, inplace=True)

In [7]:
users.head()

,vio_productos,tiene_checkouts,tiene_conversions,cant_prod_vistos
person,,,,
4886f805,True,True,False,4.0
ad93850f,True,True,False,20.0
0297fc1e,True,True,False,404.0
2d681dd8,True,True,False,13.0
cccea85e,True,True,False,739.0


In [8]:
cantidad_checkouts = df.loc[df['event'] == 'checkout']
cantidad_checkouts = cantidad_checkouts.groupby('person')['event'].value_counts().unstack()
cantidad_checkouts.rename(columns={'checkout':'cantidad_checkouts'}, inplace=True)
users = users.join(cantidad_checkouts)
users.fillna({'cantidad_checkouts':0}, inplace=True)

In [9]:
users.head()

,vio_productos,tiene_checkouts,tiene_conversions,cant_prod_vistos,cantidad_checkouts
person,,,,,
4886f805,True,True,False,4.0,1.0
ad93850f,True,True,False,20.0,1.0
0297fc1e,True,True,False,404.0,7.0
2d681dd8,True,True,False,13.0,1.0
cccea85e,True,True,False,739.0,1.0


In [10]:
cantidad_conversions = df.loc[df['event'] == 'conversion']
cantidad_conversions = cantidad_conversions.groupby('person')['event'].value_counts().unstack()
cantidad_conversions.rename(columns={'conversion':'cantidad_conversions'}, inplace=True)
users = users.join(cantidad_conversions)
users.fillna({'cantidad_conversions':0}, inplace=True)

In [11]:
users.head()

,vio_productos,tiene_checkouts,tiene_conversions,cant_prod_vistos,cantidad_checkouts,cantidad_conversions
person,,,,,,
4886f805,True,True,False,4.0,1.0,0.0
ad93850f,True,True,False,20.0,1.0,0.0
0297fc1e,True,True,False,404.0,7.0,0.0
2d681dd8,True,True,False,13.0,1.0,0.0
cccea85e,True,True,False,739.0,1.0,0.0


In [12]:
users.describe().T

,count,mean,std,min,25%,50%,75%,max
cant_prod_vistos,38829.0,32.144119,79.762927,0.0,3.0,10.0,29.0,2355.0
cantidad_checkouts,38829.0,1.682119,2.717728,0.0,1.0,1.0,2.0,197.0
cantidad_conversions,38829.0,0.182621,0.987561,0.0,0.0,0.0,0.0,129.0


In [13]:
#users.to_csv('data/features_basicas.csv', sep=',')

## Sesiones

In [14]:
sesiones = df.copy()

In [15]:
sesiones.sort_values(by='timestamp', inplace=True)
sesiones['diff'] = sesiones.groupby('person')['timestamp'].diff() / np.timedelta64(1, 'h')
sesiones['diff'].fillna(0, inplace=True)

In [16]:
sesiones[['timestamp', 'person', 'diff']].head(10)

,timestamp,person,diff
2307205,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753202,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753201,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753200,2018-01-01 08:09:44,0f4e2a4b,0.003611
1753207,2018-01-01 08:45:29,0f4e2a4b,0.595833
2307206,2018-01-01 08:45:29,0f4e2a4b,0.000000
1753210,2018-01-01 08:45:29,0f4e2a4b,0.000000
1753203,2018-01-01 08:45:43,0f4e2a4b,0.003889
1753204,2018-01-01 08:48:57,0f4e2a4b,0.053889
1753213,2018-01-01 08:49:05,0f4e2a4b,0.002222


In [17]:
sesiones['new_session'] = sesiones.groupby('person')['diff'].apply(lambda x: x > 1.0)
sesiones['session_id'] = sesiones.groupby('person')['new_session'].cumsum()

In [18]:
sesiones[['timestamp', 'person', 'event', 'diff', 'session_id']].head()

,timestamp,person,event,diff,session_id
2307205,2018-01-01 08:09:31,0f4e2a4b,visited site,0.000000,0.0
1753202,2018-01-01 08:09:31,0f4e2a4b,search engine hit,0.000000,0.0
1753201,2018-01-01 08:09:31,0f4e2a4b,ad campaign hit,0.000000,0.0
1753200,2018-01-01 08:09:44,0f4e2a4b,viewed product,0.003611,0.0
1753207,2018-01-01 08:45:29,0f4e2a4b,ad campaign hit,0.595833,0.0


In [19]:
checkouts_por_sesion = sesiones.loc[sesiones['event'] == 'checkout']

In [20]:
sesiones.groupby(['person', 'session_id'])['event'].value_counts()

person    session_id  event            
0008ed71  0.0         checkout              1
          1.0         checkout              1
                      visited site          1
          2.0         checkout              1
                      generic listing       1
                      visited site          1
00091926  0.0         viewed product        6
                      visited site          1
          1.0         viewed product        5
                      visited site          1
          2.0         viewed product       13
                      visited site          1
          3.0         viewed product        5
                      visited site          1
          4.0         viewed product       10
                      visited site          1
          5.0         viewed product        5
                      ad campaign hit       2
                      visited site          2
          6.0         brand listing         1
                      viewed product    